__for working on logging and status display tools__

In [ ]:
%cd ~/Dropbox/CanvasHacks

#Plotting 
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf')

import pandas as pd
pd.options.display.max_rows = 999


from CanvasHacks import environment


# This aren't used in the non-saving version
from CanvasHacks.FileTools import makeDataFileIterator

# Import the Canvas class
from canvasapi import Canvas
from canvasapi.quiz import QuizReport, Quiz
from canvasapi.requester import Requester
from canvasapi.conversation import Conversation

# Initialize a Canvas api objects
canvas = Canvas(environment.CONFIG.canvas_url_base, environment.CONFIG.canvas_token)

import inspect
def look_inside(obj):
    print(inspect.getmembers(obj, lambda a:not(inspect.isroutine(a))))

from CanvasHacks.PeerReviewed.Definitions import *

from CanvasHacks.Repositories.status import InvitationStatusRepository, FeedbackStatusRepository
from CanvasHacks.Repositories.quizzes import QuizRepository

from CanvasHacks.Widgets.LiveSelection import make_test_selector
from CanvasHacks.Widgets.AssignmentSelection import make_unit_chooser

from CanvasHacks.SkaaSteps.SendInitialWorkToReviewer import SendInitialWorkToReviewer

from CanvasHacks.Models.review_association import ReviewAssociation
from CanvasHacks.Models.status_record import ComplexStatusRecord, FeedbackReceivedRecord, InvitationReceivedRecord,\
    StatusRecord
from CanvasHacks.Models.student import StoredStudent

from CanvasHacks.Repositories.students import StudentRepository
from CanvasHacks.Repositories.reviewer_associations import AssociationRepository

from faker import Faker
fake = Faker()
from tests.factories.PeerReviewedFactories import discussion_entry_factory

In [ ]:
import CanvasHacks.testglobals
# Set on the global variable (which is only used in certain tests)
# CanvasHacks.testglobals.TEST
CanvasHacks.testglobals.TEST_WITH_FILE_DB = True

In [ ]:
make_test_selector()
make_unit_chooser()

# Dashboard

In [ ]:
# just for easy db access
SEND = False
step = SendInitialWorkToReviewer(course=environment.CONFIG.course, unit=environment.CONFIG.unit, send=SEND)
dao = step.dao
unit = environment.CONFIG.unit

In [ ]:

studentRepo = StudentRepository(environment.CONFIG.course)
studentRepo.download()


skaa_data = []
discussion_data = []
skaa_components = [c for c in unit.components if isinstance(c, SkaaReviewGroup)]
discussion_components = [c for c in unit.components if isinstance(c, DiscussionGroup)]


def get_data(studentRepo, components):
    invite_fields = {Review: 'invited_to_review', MetaReview: 'invited_to_metareview', DiscussionReview: 'invited_to_discussion_review'}
    feedback_fields = {Review: 'received_ca_feedback', MetaReview: 'received_meta_feedback', DiscussionReview: 'received_discussion_feedback'}

    data = []
    for sid, obj in studentRepo.data.items():
        d = { 
            'student': obj.name,
            'canvas_id': sid,
            'csun_id': obj.sis_user_id
        }
        for c in components:
            assignRepo = AssociationRepository(dao, c)
            if len(assignRepo.get_associations()) > 0:
                try:
                    # get the record where the student is the reviwer
                    a = assignRepo.get_by_reviewer(sid)
                    # get the name of the student being assessed
                    d['reviewing'] = studentRepo.get_student_name(a.assessee_id)
                    # get the record where the student is the author
                    b = assignRepo.get_by_author(sid)
                    # get the name 
                    d['reviewed_by'] = studentRepo.get_student_name(b.assessor_id)
                except AttributeError:
                    pass

            invite_fieldname = invite_fields.get(type(c))

            if invite_fieldname is not None:
                inv = InvitationStatusRepository(dao, c)
                d[invite_fieldname] = pd.to_datetime(inv.received_at(sid))

            # Note: can't do in similar way to invitations since invited to metareview and received ca feedback 
            # use different activities. The invitation is for the upcoming one which provides feedback
            # on the previous one

            if isinstance(c, InitialWork):
                # we can't use the review object because feedback on the review
                # comes from the metareview
                fb_fieldname = 'received_feedback_on_essay'
            
            if isinstance(c, Review):
                fb_fieldname = 'received_feedback_on_review'
            
            if isinstance(c, DiscussionForum):
                fb_fieldname = 'received_discussion_feedback'
#             fb_fieldname = feedback_fields.get(type(c))
            
            if fb_fieldname is not None:
                fr = FeedbackStatusRepository(dao, c)
                d[fb_fieldname] = pd.to_datetime(fr.received_at(sid))

        data.append(d)
    return data

skaa_data = get_data(studentRepo, skaa_components)
discussion_data = get_data(studentRepo, discussion_components)

skaa_data = pd.DataFrame(skaa_data)
discussion_data = pd.DataFrame(discussion_data)

# Divide up who has done initial work and been assigned a reviewer
ca = skaa_data[~skaa_data.reviewing.isnull()]
no_ca = skaa_data[skaa_data.reviewing.isnull()]

# Students whose reviewer has and has not turned in review
reviewed = ca[~ca.received_feedback_on_essay.isnull()]
nonreviewed = ca[ca.received_feedback_on_essay.isnull()].drop(['reviewing'], axis=1)

# Metareviewer turned in
metareviewed = ca[~ca.received_feedback_on_review.isnull()].drop(['reviewed_by'], axis=1)
nonmetareviewed = ca[ca.received_feedback_on_review.isnull()].drop(['reviewed_by'], axis=1)


print("===================== initial work =====================")
print("{} students have turned in essay and been paired up".format(len(ca)))
print("{} students haven't turned in essay".format(len(no_ca)))

print("===================== review =====================")
print("{} students' reviewers has turned in the review".format(len(reviewed)))
print("{} students have a reviewer who hasn't turned in the review".format(len(nonreviewed)))

print("===================== metareview =====================")
print("{} students' authors have turned in the metareview".format(len(metareviewed)))
print("{} students' authors haven't turned in the metareview".format(len(nonmetareviewed)))


In [ ]:
skaa_data
# discussion_data

## Students who have been paired up

In [ ]:
print("{} students have turned in essay and been paired up".format(len(ca)))
ca

## Students who haven't turned in the Essay / content assignment



In [ ]:
print("{} students haven't turned in essay".format(len(no_ca)))
no_ca

## Students whose reviewer hasn't turned in the review

In [ ]:
# reviewed = ca[~ca.received_ca_feedback.isnull()]
print("{} students' reviewers has turned in the review".format(len(reviewed)))
# nonreviewed = ca[ca.received_ca_feedback.isnull()].drop(['reviewing'], axis=1)
print("{} students have a reviewer who hasn't turned in the review".format(len(nonreviewed)))
nonreviewed

## Reviewers whose author hasn't turned in metareview

Here we needd to look at the reviewing field. The 'student' is the person who didn't turn in the metareview

In [ ]:
# nonmetareviewed = ca[ca.received_meta_feedback.isnull()].drop(['reviewed_by'], axis=1)
print("{} students' authors haven't turned in the metareview".format(len(nonmetareviewed)))
nonmetareviewed

In [ ]:
unit.components

In [ ]:
fr = FeedbackStatusRepository(dao, unit.review)


In [ ]:
fr.previously_received[0].__dict__

In [ ]:
fr.reviewers_with_authors_sent_feedback

In [ ]:
fr.received_at(86937)

# Discussion

## Students who haven't posted to the forum

In [ ]:
posters = discussion_data[~discussion_data.reviewing.isnull()]
nonposters = discussion_data[discussion_data.reviewing.isnull()]
nonposters

## Students whose reviewers didn't do the review

In [ ]:
posters[posters.received_discussion_feedback.isnull()]

# Nag

## Nag people to turn in Essay

In [ ]:
from CanvasHacks.Messaging.nagging import EssayNonSubmittersMessaging

messenger = EssayNonSubmittersMessaging(unit, send=True)

non_submitters = [(cid, studentRepo.get_student_first_name(cid)) for cid in no_ca.canvas_id.tolist()]
len(non_submitters)

In [ ]:
for cid, name in non_submitters:
    messenger.send_message_to_student(cid, name)

In [ ]:
non_submitters[0]

In [ ]:
invite_repos = {}
feedback_repos = {}
for c in unit.components:
    ftuple = invite_fields.get(type(c))
    print(c, ftuple)

#     invite_repos[c.title_base] = SentInvitationStatusRepository(dao, c)
#     feedback_repos[c.title_base] = SentFeedbackStatusRepository(dao, c)



In [ ]:
q = """
SELECT s.name AS student, s.id AS canvas_id, s.csun_id,
sring.name AS reviewing,
sby.name AS reviewed_by
FROM students s
-- person they are reviewing
INNER JOIN students sring ON rass.assessee_id = sring.id
INNER JOIN review_associations rass ON s.id = rass.assessor_id
-- person they are reviewed by
INNER JOIN students sby ON revby.assessor_id = sby.id
INNER JOIN review_associations revby ON s.id = revby.assessee_id
WHERE rass.activity_id = 641141 
AND revby.activity_id = 641141
"""
r = dao.session.execute(q)
r = [f for f in r]
r

In [ ]:
activity = unit.discussion_forum
all_recs = dao.session\
            .query( ReviewAssociation )\
            .join( FeedbackReceivedRecord, FeedbackReceivedRecord.student_id == ReviewAssociation.assessee_id ).all()
#             .filter( ReviewAssociation.activity_id == activity.id )\
#             .filter( FeedbackReceivedRecord.activity_id == activity.id )\
#             .all()


In [ ]:
columns = [
    'student',
    'csun_id',
    'canvas_id',
    # Assigned to review
    'reviewing',
    'invited_to_review_on',
    # who is reviewing them
    'being_reviewed_by',
    'reviewer_invited_on',
    # Feedback
    'received_feedback_on'
]

In [ ]:
# works for getting names
"""SELECT s.name AS student, s.id as canvas_id, s.csun_id, 
(select ss.name  from students ss 
inner join review_associations rc on ss.id = rc.assessor_id
where rc.assessee_id = s.id)  as reviewing, 
(select revby.name from students revby 
inner join review_associations rb on revby.id = rb.assessee_id
where rb.assessor_id = s.id)  as reviewed_by
from students s 
inner join review_associations ra on s.id = ra.assessor_id
inner join students ss  on ra.assessee_id = ss.id

"""

In [ ]:
q = """select ra.assessor_id as reviewer, ra.assessee_id as author, fb.sent_at"""

In [ ]:
"""
SELECT ra.assessee_id AS author_id, 
ra.assessor_id AS reviewer_id, 
fb.sent_at AS received_fb_date,
inv.sent_at AS invited_date
FROM review_associations as ra 
LEFT JOIN feedback_received as fb
ON ra.assessee_id = fb.student_id 
LEFT JOIN invitation_received as inv
ON ra.assessor_id = inv.student_id
"""

In [ ]:
all_recs

In [ ]:
unit.discussion_forum.id

In [ ]:
# dao.initialize_db_file()

In [ ]:
students = []
for d in studentRepo.data.values():
    s = StoredStudent(**d.attributes)
    students.append(s)
    dao.session.add(s)
dao.session.commit()

In [ ]:
students

In [ ]:
[v for v in studentRepo.data.values()][0].__dict__

In [ ]:
s = StoredStudent(short_name='t')
# s.sis_user_id = 2

In [ ]:
s.__dict__

# Logging

In [ ]:
from CanvasHacks.Logging import *

# environment.CONFIG.set_test()


In [ ]:
StudentWorkLogger.write('taco 22')

In [ ]:

# def log_student_work(func):
#     # @wraps(func)
#     def wrapper(*args, **kwargs):
#         # handle logging
#         StudentWorkLogger.write("\n".join(args))
#         # call og function
#         func(*args, **kwargs)

#     return wrapper

@log_student_work
def test(content):
    print('yay', content)

In [ ]:
test('dog')

In [ ]:
j ="d"
j += 'i'
j

In [ ]:
class T:
    
    @property
    def j(self):
        return 'k'

T.j

# Message logging

In [ ]:
MessageLogger.write('hello')

In [ ]:
@log_message
def sender(to, fromf, p):
    print('j')
    return 'l'

In [ ]:
sender(to='f', fromf=2, p='s')

In [ ]:
@log_message
def esender(to, fromf, p):
    raise Exception('taco')

In [ ]:
esender('f', 2, 's')

In [ ]:
e =Exception('r')

In [ ]:
e.__str__()

In [ ]:
# machine readable notification logs

In [ ]:
message_types = [
    # reviewer gets initial work to review
    'initial-work',
    # author of original work gets feedback
    'peer-review-feedback',
    # reviewer gets metareview on feedback
    'metareview-feedback'
]

Want to log: 

review assignment id 

    Let's me tie back to the other student

receipient id

    The student who was sent the message
    
time sent

    duh
    
type of message

    - What the content was
    
    
So when I download new data I can lok up the review assignments by the assessor id from the new data, 